In [1]:
from scipy import sparse

def gen_rand_sparse_mat(m, n, density):
    M = sparse.rand(m, n, density=density)
    M.data[:] = 1
    return M

X = gen_rand_sparse_mat(10, 10, 0.1)
print X.toarray()

[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.]]


In [2]:
TAGS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
ITEM_COUNT = 20
TAG_COUNT = len(TAGS)
DENSITY = 0.15

# X: tag matrix
# T: maximun number of questions
# GAMMA: discount factor
X = gen_rand_sparse_mat(ITEM_COUNT, TAG_COUNT, DENSITY)
T = 5
GAMMA = 0.75

print X.toarray()

[[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  1.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  1.  1.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [3]:
import numpy as np
from scipy import sparse

def questionSellectionAlgo_greedy(X, w, qs):
    X_qs = X.toarray()[:, qs]
    E = np.dot(w, X_qs) / ITEM_COUNT
    q = np.argmin(np.fabs(E - 0.5))
        
    return qs[q]
    
print questionSellectionAlgo_greedy(X, np.ones(ITEM_COUNT), range(TAG_COUNT))

0


In [4]:

def interactiveRecommandation(X, T, GAMMA):
    r_qs = np.array(range(TAG_COUNT)) # remain questions: had not asked
    w = np.ones(ITEM_COUNT)
    t = 0
    for t in range(T):
        q = questionSellectionAlgo_greedy(X, w, r_qs)
        r_qs = r_qs[r_qs != q]
        
        X_q = X.toarray()[:, q]
        print 'select', TAGS[q], ', X(q): ', X_q
        
        ans = raw_input('has ' + TAGS[q] + '? (0/1): ')
        g = np.ones(ITEM_COUNT)
        if ans == '1':
            g = X_q
        else:
            g = (1 - X_q)
        g[g == 0] = GAMMA
        w = w*g
        print 'W: ', w
    return w

interactiveRecommandation(X, T, GAMMA)

select A , X(q):  [ 0.  0.  0.  0.  0.  1.  1.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.
  0.  0.]
has A? (0/1): 1
W:  [ 0.75  0.75  0.75  0.75  0.75  1.    1.    0.75  1.    0.75  0.75  1.
  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75]
select B , X(q):  [ 0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  0.  1.]
has B? (0/1): 1
W:  [ 0.5625  0.75    0.5625  0.5625  0.5625  0.75    0.75    0.5625  1.
  0.5625  0.5625  0.75    0.5625  0.75    0.5625  0.5625  0.5625  0.5625
  0.5625  0.75  ]
select D , X(q):  [ 0.  1.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
  0.  0.]
has D? (0/1): 1
W:  [ 0.421875  0.75      0.421875  0.421875  0.5625    0.5625    0.75
  0.421875  0.75      0.421875  0.421875  0.5625    0.421875  0.5625
  0.421875  0.421875  0.5625    0.421875  0.421875  0.5625  ]
select C , X(q):  [ 0.  1.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
has C? (0/1): 1
W:  [ 0.31640625  0.75        0.31640625 

array([ 0.31640625,  0.5625    ,  0.23730469,  0.31640625,  0.31640625,
        0.31640625,  0.5625    ,  0.23730469,  0.421875  ,  0.31640625,
        0.23730469,  0.31640625,  0.23730469,  0.31640625,  0.31640625,
        0.31640625,  0.31640625,  0.23730469,  0.23730469,  0.31640625])